# #30DayChartChallenge
**5th edition - April 2025**

In [ ]:
# Imports
import os
import seaborn
import matplotlib.pyplot as plt
import geopandas as gpd
import leafmap
from owslib.wfs import WebFeatureService

# 11th of April 2025
**Day 11 | Distributions": Stripes**


In [ ]:
# Get the data
wfs_icf = "http://geoserver.icf.gob.hn/icfpub/wfs"
wfs = WebFeatureService(wfs_icf, version="1.1.0")
# Get the list of layers
layers = wfs.contents

# # Print the layers
# print("Available layers:")
# for layer in layers:
#     print(f"- {layer}")

In [ ]:
# Get the Microcuenca layer
layer_name = "icfpub:microcuencas_declaradas_gp"

# Get the data
response = wfs.getfeature(typename=layer_name, outputFormat="application/json")

gdf_mc = gpd.read_file(response)

In [ ]:
# Explore the data
# print("Microcuenca data:")
# print(gdf_mc.head())
print("Microcuenca data types:")
print(gdf_mc.dtypes)
print("Microcuenca data columns:")
print(gdf_mc.columns)

In [ ]:
gdf_mc["area_ha"].head()

In [ ]:
# aggregate area_ha by year
df = gdf_mc.groupby(by=["anio_decla"])["area_ha"].sum().reset_index(name="area_ha")
df = df.query("anio_decla >= 2000")
# # Print the data
# print("Microcuenca data by year:")
# print(df.head())

In [ ]:
df

In [ ]:
# Normalize the area for the colormap (0-1)
norm_area = (df["area_ha"] - df["area_ha"].min()) / (
    df["area_ha"].max() - df["area_ha"].min()
)

# Create the striped plot
fig, ax = plt.subplots(figsize=(12, 3))  # Adjusted height for better readability

# Draw a bar for each year with color based on the normalized area
bars = ax.bar(
    df["anio_decla"],  # X-axis
    height=1,  # Fixed height (no Y-scale)
    width=1,  # Bar width
    color=plt.cm.Reds(norm_area),  # Colormap mapping
    edgecolor="none",  # No borders
)

# Aesthetic adjustments
ax.set_xlim(df["anio_decla"].min() - 0.5, df["anio_decla"].max() + 0.5)
ax.set_ylim(0, 1)
ax.set_xticks(df["anio_decla"])  # Add ticks for each year
ax.set_xticklabels(
    df["anio_decla"], rotation=45, fontsize=8
)  # Rotate and format labels
ax.tick_params(axis="x", which="both", length=0)  # Hide tick marks but keep labels
ax.spines["bottom"].set_visible(False)  # Hide the bottom spine
ax.spines["top"].set_visible(False)  # Hide the top spine
ax.spines["left"].set_visible(False)  # Hide the left spine
ax.spines["right"].set_visible(False)  # Hide the right spine

# Add a title
ax.set_title(
    "Microcuencas declaradas por año\nIntensidad = Área en hectáreas",
    pad=20,
    loc="center",
    fontsize=14,
    fontweight="bold",
)

# Add a colorbar explicitly tied to the colormap
sm = plt.cm.ScalarMappable(
    cmap="Reds", norm=plt.Normalize(vmin=df["area_ha"].min(), vmax=df["area_ha"].max())
)
sm.set_array([])  # Required to avoid errors
cbar = fig.colorbar(sm, ax=ax, orientation="horizontal", pad=0.2)
cbar.set_label("Área (hectáreas)", fontsize=10)

# Add a footer with the source, challenge information, and author details
fig.text(
    0.5,
    -0.1,  # Position below the plot
    "Fuente: Instituto de Conservación Forestal (ICF) | #30DayChartChallenge - Día 11: Distributions:Stripes\nAutor: Luis López, @lalgonzales",
    ha="center",
    fontsize=10,
    color="gray",
)

# Adjust layout

plt.tight_layout()
char_out = "results/microcuencas_declaradas_por_anio.png"
if not os.path.exists("results"):
    os.makedirs("results")
plt.savefig(char_out, dpi=300, bbox_inches="tight")
plt.show()